## Transform Minmax Results
Please place the file containing the aggregated results data in the appropriate location.

In [ ]:
import sys

In [ ]:
import polars as pl
import pandas as pd
import numpy as np

np.random.seed(777)

In [ ]:
sys.path.append(f"./main_scripts")

In [ ]:
from io_utils import get_accuracy, get_recall

In [ ]:
sp=[0,3,6,9]

## Pre-step: Transform Minmax Results
Please place the Minmax results in the same folder as this notebook.

In [ ]:
for ai_name in ["g100", "f100", "l250", "mix"]:
    for num_ai in sp:
        for iter in range(5):
            df = pl.read_csv(f"minmax_results_raw/result_{ai_name}_{num_ai}_{iter+1}.csv", has_header=False, dtypes=[pl.Utf8],
                            new_columns=["task","1","2","3","4","5","6"])
            df = df.with_columns(
                max_val = pl.max_horizontal(["1","2","3","4","5","6"])
            ).with_columns(
                label = (
                    pl.when(pl.col("1") == pl.col("max_val")).then(pl.lit("0"))
                    .when(pl.col("2") == pl.col("max_val")).then(pl.lit("1"))
                    .when(pl.col("3") == pl.col("max_val")).then(pl.lit("2"))
                    .when(pl.col("4") == pl.col("max_val")).then(pl.lit("3"))
                    .when(pl.col("5") == pl.col("max_val")).then(pl.lit("4"))
                    .otherwise(pl.lit("5"))
                )
            ).drop("max_val")
            df = df.select(["task","label"])
            if ai_name == "mix":
                num_ai_fixed = num_ai // 3
            else:
                num_ai_fixed = num_ai
            df.write_csv(f"main_scripts/ds1task5/workdir/results/minmax_{ai_name}_AI={num_ai_fixed}_{iter}.csv")

## Main-step: Get Scores
Please put the results data in `./`

In [ ]:
TARGET = "ds1task5"
LABELS = {
    0: "section_230",
    1: "trump_ban",
    2: "twitter_support",
    3: "platform_polices",
    4: "complaint",
    5: "other",
}
EXCEPT_LABELS = [5,]

In [ ]:
human = pl.read_csv("datasets/human.csv")
gt = human.select(["task", "gt"]).unique().sort("task").to_pandas()

In [ ]:
n_classes = 6
with open("ds1task5_aggregation_results_minmax.csv", "w") as f:
        header = "ai_name,num_ai,method,iter,accuracy,"
        f.write(header)
        for key in LABELS.keys():
            if key in EXCEPT_LABELS:
                continue
            f.write(f"recall_{LABELS[key]},")
        for i in range(n_classes):
            f.write(f"uc_pih_{i},")
        for i in range(n_classes-1):
            f.write(f"uc_pia_{i},")
        f.write(f"uc_pia_{i+1}")
        f.write("\n")
        for ai_name in ["g100", "f100", "l250", "mix"]:
            for num_ai in sp:
                num_ai_fixed = num_ai // 3 if ai_name == "mix" else num_ai
                for iter in range(5):
                    df = pl.read_csv(f"main_scripts/ds1task5/workdir/results/minmax_{ai_name}_AI={num_ai_fixed}_{iter}.csv")
                    ret = df.to_pandas().set_index("task")
                    acc = get_accuracy(ret, gt.set_index("task"))
                    recalls = []
                    for key in LABELS.keys():
                        if key in EXCEPT_LABELS:
                            continue
                        recall = get_recall(
                            ret,
                            gt.set_index("task"),
                            biased_tasks=gt[gt["gt"]==key]["task"].to_list()
                        )
                        recalls.append(recall)
                    uc_text = ",".join(["-1" for _ in range(n_classes*2)])
                    # write results
                    f.write(f"{ai_name},{num_ai_fixed},minmax,{iter},{acc},")
                    for recall in recalls:
                        f.write(f"{recall},")
                    f.write(f"{uc_text}\n")
                    f.flush()